In [ ]:
# GPU required for this notebooke: used A100

In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers datasets detoxify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
rag_dir = '/content/drive/My Drive/kilt_rag_data'

Mounted at /content/drive


In [ ]:
import json
import os
import numpy as np
from sentence_transformers import SentenceTransformer # Assuming embedder is already d

In [ ]:
embeddings_dir = os.path.join(rag_dir, "embeddings")
os.makedirs(embeddings_dir, exist_ok=True)

paras_dir = os.path.join(rag_dir, "paras")
os.makedirs(paras_dir, exist_ok=True)

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2", device='cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#embedder = SentenceTransformer("BAAI/bge-small-en-v1.5", device='cuda')
embedding_batch_size = 512   #512
para_chunk_size = 5000     #5000

In [ ]:
import os
import json
import pickle
from tqdm import tqdm

# === Helper ===
def chunked(data, size):
    for i in range(0, len(data), size):
        yield data[i:i + size]

def embed_and_save(paragraphs, chunk_id):
    all_embeddings = []
    for batch in tqdm(chunked(paragraphs, para_chunk_size), desc=f"Embedding chunk {chunk_id}", leave=False):
        batch_embs = embedder.encode(
            batch,
            batch_size=embedding_batch_size,
            convert_to_numpy=True,
            show_progress_bar=False
        )
        all_embeddings.append(batch_embs)
    embeddings = np.vstack(all_embeddings)
    # out_path = os.path.join(embeddings_dir, f"kilt_embeddings_chunk_{chunk_id}.npz")
    # np.savez_compressed(out_path, embeddings)

    out_path_f16 = os.path.join(embeddings_dir, f"kilt_embeddings_chunk_{chunk_id}_fp16.npz")
    embeddings_f16 = embeddings.astype(np.float16)
    np.savez_compressed(out_path_f16, embeddings_f16)

    print(f"Saved embeddings: {out_path_f16}")

In [ ]:
# === Process each paragraph file ===
chunk_files = sorted([f for f in os.listdir(paras_dir) if f.endswith(".pkl")])[:20]
for chunk_file in chunk_files:
    print(f"Processing {chunk_file}")
    chunk_id = int(chunk_file.split("_")[-1].split(".")[0])
    with open(os.path.join(paras_dir, chunk_file), "rb") as f:
        paragraphs = pickle.load(f)
    print(f"\n Embedding paragraph chunk {chunk_id} ({len(paragraphs)} paragraphs)")
    embed_and_save(paragraphs, chunk_id)

Processing kilt_paras_chunk_0.pkl

 Embedding paragraph chunk 0 (1978344 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_0_fp16.npz
Processing kilt_paras_chunk_1.pkl

 Embedding paragraph chunk 1 (1933434 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_1_fp16.npz
Processing kilt_paras_chunk_10.pkl

 Embedding paragraph chunk 10 (1879356 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_10_fp16.npz
Processing kilt_paras_chunk_11.pkl

 Embedding paragraph chunk 11 (1863916 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_11_fp16.npz
Processing kilt_paras_chunk_12.pkl

 Embedding paragraph chunk 12 (1900075 paragraphs)


...

Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_22_fp16.npz
Processing kilt_paras_chunk_23.pkl

 Embedding paragraph chunk 23 (1898844 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_23_fp16.npz
Processing kilt_paras_chunk_24.pkl

 Embedding paragraph chunk 24 (1814565 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_24_fp16.npz
Processing kilt_paras_chunk_25.pkl

 Embedding paragraph chunk 25 (1927712 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_25_fp16.npz
Processing kilt_paras_chunk_26.pkl

 Embedding paragraph chunk 26 (1976456 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_26_fp16.npz


In [ ]:
# I'm using all-MiniLM-L6-v2, which returns:
# Shape: [1_978_344 x 384]
# Type: float32 = 4 bytes
# Size: 1_978_344M * 384 * 4 = ~2.5 GB to 3 GB (theoretical)
# File compression (.npz) might raise it due to overhead or metadata

In [ ]:
# === Process each paragraph file 20 to 40 chunk===
chunk_files = sorted([f for f in os.listdir(paras_dir) if f.endswith(".pkl")])[20:40]
for chunk_file in chunk_files:
      print(f"Processing {chunk_file}")
      chunk_id = int(chunk_file.split("_")[-1].split(".")[0])
      with open(os.path.join(paras_dir, chunk_file), "rb") as f:
          paragraphs = pickle.load(f)
      print(f"\n Embedding paragraph chunk {chunk_id} ({len(paragraphs)} paragraphs)")
      embed_and_save(paragraphs, chunk_id)

Processing kilt_paras_chunk_27.pkl

 Embedding paragraph chunk 27 (1936681 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_27_fp16.npz
Processing kilt_paras_chunk_28.pkl

 Embedding paragraph chunk 28 (1994416 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_28_fp16.npz
Processing kilt_paras_chunk_29.pkl

 Embedding paragraph chunk 29 (1857387 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_29_fp16.npz
Processing kilt_paras_chunk_3.pkl

 Embedding paragraph chunk 3 (2002378 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_3_fp16.npz
Processing kilt_paras_chunk_30.pkl

 Embedding paragraph chunk 30 (1922860 paragraphs)


...

Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_40_fp16.npz
Processing kilt_paras_chunk_41.pkl

 Embedding paragraph chunk 41 (2065169 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_41_fp16.npz
Processing kilt_paras_chunk_42.pkl

 Embedding paragraph chunk 42 (1838265 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_42_fp16.npz
Processing kilt_paras_chunk_43.pkl

 Embedding paragraph chunk 43 (1879717 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_43_fp16.npz
Processing kilt_paras_chunk_44.pkl

 Embedding paragraph chunk 44 (1941796 paragraphs)


Saved embeddings: /content/drive/My Drive/kilt_rag_data/embeddings/kilt_embeddings_chunk_44_fp16.npz


In [ ]:
# === Process each paragraph file 40 to last chunk===
chunk_files = sorted([f for f in os.listdir(paras_dir) if f.endswith(".pkl")])[40:]
for chunk_file in chunk_files:
      print(f"Processing {chunk_file}")
      chunk_id = int(chunk_file.split("_")[-1].split(".")[0])
      with open(os.path.join(paras_dir, chunk_file), "rb") as f:
          paragraphs = pickle.load(f)
      print(f"\n Embedding paragraph chunk {chunk_id} ({len(paragraphs)} paragraphs)")
      embed_and_save(paragraphs, chunk_id)